# **Interactive 3D Visualization Framework For Machine Learning Based Network Intrusion Detection Systems**

# Libraries



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import warnings
from math import log2
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.feature_selection import SelectKBest, f_classif, RFECV, RFE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier,AdaBoostClassifier, BaggingClassifier,VotingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import mutual_info_classif
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight


# Enable inline plotting for Jupyter notebooks
%matplotlib inline

# Filter warnings
warnings.filterwarnings('ignore')

# Dataset Load

In [ ]:
df = pd.read_csv('NSL_KDD.csv')

# Pipeline Implementation

In [ ]:

# step 1.1: Safe fallback to keep this cell runnable
COL_LABEL = "label"
_need_fallback = False
try:
    df
    COL_LABEL
except NameError:
    _need_fallback = True

if _need_fallback:
    rng = np.random.RandomState(42)
    n = 300
    df = pd.DataFrame({
        "num1": rng.randn(n),
        "num2": rng.rand(n) * 5,
        "cat1": rng.choice(["tcp", "udp", "icmp"], size=n),
        "cat2": rng.choice(["low", "med", "high"], size=n),
        "label": ((rng.randn(n) + rng.rand(n) * 2) > 1).astype(int)
    })


# Step 1.2: Identify feature matrix and target BEFORE split
X_all = df.drop(columns=[COL_LABEL]).copy()
y_all = df[COL_LABEL].copy()

# find column types from the full dataset
cat_cols = X_all.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X_all.select_dtypes(include=[np.number]).columns.tolist()

# Step 1.3: One Hot on categoricals and numeric pass through, done BEFORE split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

# encode categoricals on the full data
if len(cat_cols) > 0:
    X_all_cat = ohe.fit_transform(X_all[cat_cols].astype(str))
    ohe_cols = ohe.get_feature_names_out(cat_cols).tolist()
else:
    X_all_cat = np.empty((len(X_all), 0))
    ohe_cols = []

# numeric part
X_all_num = X_all[num_cols].to_numpy(dtype=float) if len(num_cols) > 0 else np.empty((len(X_all), 0))

# combine
X_all_oh = np.hstack([X_all_num, X_all_cat])
feat_names = num_cols + ohe_cols

print("After One Hot shapes:", X_all_oh.shape)

After One Hot shapes: (185559, 123)


In [ ]:
# split AFTER encoding
X_train_oh, X_test_oh, y_train, y_test = train_test_split(
    X_all_oh, y_all, test_size=0.25, random_state=42, stratify=y_all if y_all.nunique() > 1 else None
)

In [ ]:
# STEP 2: Normalization with MinMaxScaler
# ================================
scaler =  StandardScaler()
X_train_normalized = scaler.fit_transform(X_train_oh)
X_test_normalized  = scaler.transform(X_test_oh)

print("After Normalization shapes:", X_train_normalized.shape, X_test_normalized.shape)

After Normalization shapes: (139169, 123) (46390, 123)


In [ ]:
# ================================
# STEP 3.1: Training set information gain calculation
# ================================
mi = mutual_info_classif(X_train_normalized, np.asarray(y_train), random_state=42, discrete_features=False)

# 3.2: Convert scores to weights in [0,1], avoid exact zeros
mi_max = np.max(mi) if np.max(mi) > 0 else 1.0
weights = mi / mi_max
weights = np.where(weights == 0, 1e-6, weights)

In [ ]:
# ================================
# STEP 4: Weighted Transform each feature
# ================================
X_train_weighted = X_train_normalized * weights
X_test_weighted  = X_test_normalized  * weights

In [ ]:
# STEP 5: Transform features to zero mean
# ================================
train_means = X_train_weighted.mean(axis=0)
X_train_centered = X_train_weighted - train_means
X_test_centered  = X_test_weighted  - train_means

In [ ]:

# 6.1: PCA on SFS output to explain at least 95 percent variance
print("\n=== PCA output ===")
pca_probe = PCA().fit(X_train_centered)
cum_var = np.cumsum(pca_probe.explained_variance_ratio_)
n_components = int(np.argmax(cum_var >= 0.99) + 1)
pca = PCA(n_components=max(1, n_components)).fit(X_train_centered)
X_train_pca = pca.transform(X_train_centered)
X_test_pca  = pca.transform(X_test_centered)
print(f'Number of components that explain 99.0% variance: {pca.n_components_}')


# 6.2: LDA output

if len(np.unique(y_train)) > 1:
    print("\n=== LDA  output ===")
    n_classes = len(np.unique(y_train))
    n_comp_lda = min(max(1, n_classes - 1), X_train_centered.shape[1])
    lda = LDA(n_components=n_comp_lda).fit(X_train_pca, y_train)
    X_train_lda = lda.transform(X_train_pca)
    X_test_lda  = lda.transform(X_test_pca)
    print(f"LDA components used: {n_comp_lda}")






=== PCA output ===
Number of components that explain 99.0% variance: 19

=== LDA  output ===
LDA components used: 4


# ML Model Results Storage Framework

In [ ]:
# =============================================================================
# ML MODEL RESULTS STORAGE FRAMEWORK
# =============================================================================

# Creating holders to store the model performance results
ML_Model = []
ML_Config = []
accuracy = []
f1_score = []
recall = []
precision = []
auc_roc = []  # Adding a holder for AUC-ROC

# Function to call for storing the results
def storeResults(model, config, a, b, c, d, e):
    """
    Store model performance results

    Parameters:
    model: Name of the ML model
    config: Configuration name (preprocessing steps applied)
    a: Accuracy score
    b: F1 score
    c: Recall score
    d: Precision score
    e: AUC-ROC score
    """
    ML_Model.append(model)
    ML_Config.append(config)
    accuracy.append(round(a, 6))
    f1_score.append(round(b, 6))
    recall.append(round(c, 6))
    precision.append(round(d, 6))
    auc_roc.append(round(e, 6))

# Function to display and save results
def displayAndSaveResults(filename_prefix='model_results'):
    """
    Create dataframe from results, display, and save to CSV

    Parameters:
    filename_prefix: Prefix for the CSV filenames
    """
    # Creating the dataframe
    result = pd.DataFrame({
        'ML Model': ML_Model,
        'Configuration': ML_Config,
        'Accuracy': [f"{acc * 100:.3f}%" for acc in accuracy],
        'F1 Score': [f"{f1 * 100:.3f}%" for f1 in f1_score],
        'Recall': [f"{rec * 100:.3f}%" for rec in recall],
        'Precision': [f"{prec * 100:.3f}%" for prec in precision],
        'ROC_AUC': [f"{roc * 100:.3f}%" for roc in auc_roc],
    })

    # Remove duplicates if any
    result.drop_duplicates(subset=["ML Model", "Configuration"], inplace=True)

    print("\n" + "="*100)
    print("MODEL PERFORMANCE RESULTS")
    print("="*100)
    print(result.to_string(index=False))

    # Saving the result to a CSV file
    result.to_csv(f'{filename_prefix}.csv', index=False)
    print(f"\nResults saved to {filename_prefix}.csv")

    # Sorting the dataframe on accuracy and F1 Score
    sorted_result = result.sort_values(by=['Accuracy', 'F1 Score'], ascending=False).reset_index(drop=True)

    print("\n" + "="*100)
    print("SORTED MODEL PERFORMANCE RESULTS (by Accuracy and F1 Score)")
    print("="*100)
    print(sorted_result.to_string(index=False))

    # Saving the sorted result to a CSV file
    sorted_result.to_csv(f'sorted_{filename_prefix}.csv', index=False)
    print(f"\nSorted results saved to sorted_{filename_prefix}.csv")

    return result, sorted_result

# Function to clear results (useful when running multiple experiments)
def clearResults():
    """Clear all stored results"""
    global ML_Model, ML_Config, accuracy, f1_score, recall, precision, auc_roc
    ML_Model.clear()
    ML_Config.clear()
    accuracy.clear()
    f1_score.clear()
    recall.clear()
    precision.clear()
    auc_roc.clear()
    print("Results cleared!")

# Function to plot model comparison
def plotModelComparison(result_df):
    """
    Create visualization comparing model performances

    Parameters:
    result_df: DataFrame with model results
    """
    # Convert percentage strings back to floats for plotting
    metrics_cols = ['Accuracy', 'F1 Score', 'Recall', 'Precision', 'ROC_AUC']
    plot_df = result_df.copy()

    for col in metrics_cols:
        plot_df[col] = plot_df[col].str.rstrip('%').astype(float)

    # Create subplot for each metric
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.ravel()

    for idx, metric in enumerate(metrics_cols):
        # Group by model and get mean performance across configurations
        model_performance = plot_df.groupby('ML Model')[metric].mean().sort_values(ascending=False)

        # Create bar plot
        ax = axes[idx]
        bars = ax.bar(range(len(model_performance)), model_performance.values,
                      color=plt.cm.Blues(np.linspace(0.4, 0.9, len(model_performance))))
        ax.set_xticks(range(len(model_performance)))
        ax.set_xticklabels(model_performance.index, rotation=45, ha='right')
        ax.set_ylabel(f'{metric} (%)')
        ax.set_title(f'Average {metric} by Model', fontweight='bold')
        ax.grid(axis='y', alpha=0.3)

        # Add value labels on bars
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.1f}%', ha='center', va='bottom')

    # Hide the last subplot if we have 5 metrics
    if len(metrics_cols) == 5:
        axes[5].set_visible(False)

    plt.suptitle('Model Performance Comparison', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()

print("Model results storage framework loaded successfully!")
print("Available functions:")
print("- storeResults(model, config, accuracy, f1, recall, precision, auc_roc)")
print("- displayAndSaveResults(filename_prefix='model_results')")
print("- clearResults()")
print("- plotModelComparison(result_df)")

Model results storage framework loaded successfully!
Available functions:
- storeResults(model, config, accuracy, f1, recall, precision, auc_roc)
- displayAndSaveResults(filename_prefix='model_results')
- clearResults()
- plotModelComparison(result_df)


#SVM

In [ ]:
# Configuration list to store different data setups
configurations = []

configurations.append((f'PCA', X_train_pca, X_test_pca, y_train))
configurations.append((f'LDA', X_train_lda, X_test_lda, y_train))

# Step 7: Run SVM  on different configurations
print("\n=== SVM Model Performance  ===")
svm = SVC(
    kernel="rbf",
    C=1.0,
    gamma="scale",
    probability=True,
    random_state=42
    )
for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning SVM with {name} configuration...")

    svm.fit(X_train_cfg, y_train_cfg)

    y_train_pred = svm.predict(X_train_cfg)
    y_test_pred  = svm.predict(X_test_cfg)

    y_train_proba = svm.predict_proba(X_train_cfg)
    y_test_proba  = svm.predict_proba(X_test_cfg)


    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_pred),
            metrics.accuracy_score(y_test,      y_test_pred),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.f1_score(y_test,      y_test_pred,  average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.recall_score(y_test,      y_test_pred,  average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.precision_score(y_test,      y_test_pred,  average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test),      y_test_proba,  multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nSVM Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_proba, multi_class='ovr', average='macro')
    storeResults('Support Vector Machine', name,
                 metrics.accuracy_score(y_test, y_test_pred),
                 metrics.f1_score(y_test, y_test_pred, average='macro'),
                 metrics.recall_score(y_test, y_test_pred, average='macro'),
                 metrics.precision_score(y_test, y_test_pred, average='macro'),
                 auc_score)



=== SVM Model Performance  ===

Running SVM with PCA configuration...

SVM Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  0.986958  0.785539 0.788589   0.965700 0.998706
    Test  0.987088  0.773726 0.782342   0.765859 0.998304

Running SVM with LDA configuration...

SVM Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  0.954624  0.737193 0.748722   0.727092 0.994087
    Test  0.955702  0.737363 0.747554   0.728329 0.994134


In [ ]:
# === Confusion matrices for all configurations (safe to run anytime) ===
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
import numpy as np, pandas as pd

def nslkdd_confusion_table(y_true, y_pred, class_order=None, title_no=5):
    if class_order is None:
        classes = list(pd.unique(pd.Series(list(y_true) + list(y_pred))))
    else:
        present = set(pd.unique(pd.Series(list(y_true) + list(y_pred))))
        classes = [c for c in class_order if c in present] or list(present)

    cm = confusion_matrix(y_true, y_pred, labels=classes)
    df = pd.DataFrame(cm, index=[f"Actual  {c}" for c in classes], columns=classes)

    row_tot = cm.sum(axis=1)
    recalls = np.divide(np.diag(cm), row_tot, out=np.zeros_like(row_tot, dtype=float), where=row_tot != 0) * 100.0
    df["Recall (%)"] = np.round(recalls, 1)

    col_tot = cm.sum(axis=0)
    precisions = np.divide(np.diag(cm), col_tot, out=np.zeros_like(col_tot, dtype=float), where=col_tot != 0) * 100.0
    prec_row = pd.Series(np.round(precisions, 1), index=classes, name="Precision (%)")
    prec_row["Recall (%)"] = ""
    df = pd.concat([df, prec_row.to_frame().T], axis=0)

    print(f"\nTable {title_no}")
    print("Multi category classification confusion matrix for the NSL KDD dataset.\n")
    print("Predicted\n")
    print(df.to_string())
    return df

# --- Use cached predictions if available; otherwise rebuild them now ---
if 'y_pred_cache' not in globals() or not isinstance(y_pred_cache, dict) or len(y_pred_cache) == 0:
    y_pred_cache = {}
    for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
        clf = SVC(kernel="rbf", C=1.0, gamma="scale", probability=True, random_state=42)
        clf.fit(X_train_cfg, y_train_cfg)
        y_pred_cache[name] = clf.predict(X_test_cfg)

# --- Build & save confusion matrices for all configs ---
nsl_order = ["DoS", "Normal", "Probe", "R2L", "U2R"]
for name, y_pred in y_pred_cache.items():
    print(f"\n=== Confusion Matrix: {name} ===")
    tbl = nslkdd_confusion_table(y_test, y_pred, class_order=nsl_order, title_no=5)
    path = f"nslkdd_confusion_table_SVM_{name}.csv"
    tbl.to_csv(path, index=True)
    print(f"Saved to {path}")


KeyboardInterrupt: 

# Random Forest

In [ ]:
# Configuration list to store different data setups
configurations = []

configurations.append((f'PCA', X_train_pca, X_test_pca, y_train))
configurations.append((f'LDA', X_train_lda, X_test_lda, y_train))

# Step 7: Run RF different configurations
print("\n=== Random Forestr Model Performance ===")

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1
)
for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning Random Forest with {name} configuration...")

    rf.fit(X_train_cfg, y_train_cfg)

    y_train_pred = rf.predict(X_train_cfg)
    y_test_pred  = rf.predict(X_test_cfg)

    y_train_proba = rf.predict_proba(X_train_cfg)
    y_test_proba  = rf.predict_proba(X_test_cfg)


    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_pred),
            metrics.accuracy_score(y_test,      y_test_pred),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.f1_score(y_test,      y_test_pred,  average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.recall_score(y_test,      y_test_pred,  average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.precision_score(y_test,      y_test_pred,  average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test),      y_test_proba,  multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nRF Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_proba, multi_class='ovr', average='macro')
    storeResults('Random Forest', name,
                 metrics.accuracy_score(y_test, y_test_pred),
                 metrics.f1_score(y_test, y_test_pred, average='macro'),
                 metrics.recall_score(y_test, y_test_pred, average='macro'),
                 metrics.precision_score(y_test, y_test_pred, average='macro'),
                 auc_score)




=== Random Forestr Model Performance ===

Running Random Forest with PCA configuration...

RF Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  1.000000  1.000000 1.000000   1.000000 1.000000
    Test  0.998232  0.966739 0.956347   0.978985 0.999904

Running Random Forest with LDA configuration...

RF Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  1.000000  1.000000 1.000000   1.000000 1.000000
    Test  0.994546  0.967677 0.961053   0.975118 0.997484


 # KNN

In [ ]:
# Configuration list to store different data setups
configurations = []

configurations.append((f'PCA', X_train_pca, X_test_pca, y_train))
configurations.append((f'LDA', X_train_lda, X_test_lda, y_train))


# Step 7: Run KNN  on different configurations
print("\n=== KNN Model Performance ===")

knn = KNeighborsClassifier(
    n_neighbors=11,
    weights="distance",
    metric="minkowski",
    p=2,
    n_jobs=-1
)
for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning KNN with {name} configuration...")

    knn.fit(X_train_cfg, y_train_cfg)

    y_train_pred = knn.predict(X_train_cfg)
    y_test_pred  = knn.predict(X_test_cfg)

    y_train_proba = knn.predict_proba(X_train_cfg)
    y_test_proba  = knn.predict_proba(X_test_cfg)


    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_pred),
            metrics.accuracy_score(y_test,      y_test_pred),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.f1_score(y_test,      y_test_pred,  average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.recall_score(y_test,      y_test_pred,  average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.precision_score(y_test,      y_test_pred,  average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test),      y_test_proba,  multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nKNN Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_proba, multi_class='ovr', average='macro')
    storeResults('KNN', name,
                 metrics.accuracy_score(y_test, y_test_pred),
                 metrics.f1_score(y_test, y_test_pred, average='macro'),
                 metrics.recall_score(y_test, y_test_pred, average='macro'),
                 metrics.precision_score(y_test, y_test_pred, average='macro'),
                 auc_score)





=== KNN Model Performance ===

Running KNN with PCA configuration...

KNN Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training   1.00000  1.000000 1.000000   1.000000 1.000000
    Test   0.99737  0.970783 0.964408   0.977984 0.994882

Running KNN with LDA configuration...

KNN Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  1.000000  1.000000 1.000000   1.000000 1.000000
    Test  0.993684  0.958819 0.951677   0.967244 0.994301


# Gradient Boosting

In [ ]:
# Configuration list to store different data setups
configurations = []

configurations.append((f'PCA', X_train_pca, X_test_pca, y_train))
configurations.append((f'LDA', X_train_lda, X_test_lda, y_train))

# Step 7: Running Gradient Boosting  on different configurations
print("\n=== Gradient Boosting Model Performance ===")

gbc = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=1,
    subsample=1.0,
    random_state=42
)
for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning Gradient Boosting with {name} configuration...")

    gbc.fit(X_train_cfg, y_train_cfg)

    y_train_pred = gbc.predict(X_train_cfg)
    y_test_pred  = gbc.predict(X_test_cfg)

    y_train_proba = gbc.predict_proba(X_train_cfg)
    y_test_proba  = gbc.predict_proba(X_test_cfg)


    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_pred),
            metrics.accuracy_score(y_test,      y_test_pred),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.f1_score(y_test,      y_test_pred,  average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.recall_score(y_test,      y_test_pred,  average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.precision_score(y_test,      y_test_pred,  average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test),      y_test_proba,  multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nGradient Boosting Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_proba, multi_class='ovr', average='macro')
    storeResults('Gradient Boosting', name,
                 metrics.accuracy_score(y_test, y_test_pred),
                 metrics.f1_score(y_test, y_test_pred, average='macro'),
                 metrics.recall_score(y_test, y_test_pred, average='macro'),
                 metrics.precision_score(y_test, y_test_pred, average='macro'),
                 auc_score)




=== Gradient Boosting Model Performance ===

Running Gradient Boosting with PCA configuration...

Gradient Boosting Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  0.993310  0.887161 0.865444   0.935985 0.974450
    Test  0.990645  0.878778 0.863546   0.908042 0.979998

Running Gradient Boosting with LDA configuration...

Gradient Boosting Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  0.968995  0.853290 0.832289   0.902218 0.968327
    Test  0.966415  0.840327 0.823272   0.880442 0.958179


#AdaBoosting

In [ ]:
# Configuration list to store different data setups
configurations = []

configurations.append((f'PCA', X_train_pca, X_test_pca, y_train))
configurations.append((f'LDA', X_train_lda, X_test_lda, y_train))

# Step 7: Run AdaBoost  on different configurations
print("\n=== AdaBoost Model Performance ===")

ada = AdaBoostClassifier(
        estimator=DecisionTreeClassifier(max_depth=1, random_state=42),
        n_estimators=200,
        learning_rate=0.1,
        algorithm="SAMME",
        random_state=42
    )

for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning AdaBoost with {name} configuration...")

    ada.fit(X_train_cfg, y_train_cfg)

    y_train_pred = ada.predict(X_train_cfg)
    y_test_pred  = ada.predict(X_test_cfg)

    y_train_proba = ada.predict_proba(X_train_cfg)
    y_test_proba  = ada.predict_proba(X_test_cfg)


    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_pred),
            metrics.accuracy_score(y_test,      y_test_pred),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.f1_score(y_test,      y_test_pred,  average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.recall_score(y_test,      y_test_pred,  average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.precision_score(y_test,      y_test_pred,  average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test),      y_test_proba,  multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nAdaBoosting Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_proba, multi_class='ovr', average='macro')
    storeResults('AdaBoosting', name,
                 metrics.accuracy_score(y_test, y_test_pred),
                 metrics.f1_score(y_test, y_test_pred, average='macro'),
                 metrics.recall_score(y_test, y_test_pred, average='macro'),
                 metrics.precision_score(y_test, y_test_pred, average='macro'),
                 auc_score)




=== AdaBoost Model Performance ===

Running AdaBoost with PCA configuration...

AdaBoosting Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  0.889861  0.617671 0.590525   0.661199 0.970492
    Test  0.890494  0.619279 0.592466   0.661399 0.970474

Running AdaBoost with LDA configuration...

AdaBoosting Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  0.903944  0.642798 0.630292   0.658439 0.978965
    Test  0.902824  0.640876 0.628797   0.655935 0.978678


# XGBoosting


In [ ]:
# Configuration list to store different data setups
configurations = []

le = LabelEncoder()
# Fit on the union to avoid unseen class errors if test has a class not in train
le.fit(pd.concat([y_train.astype(str), y_test.astype(str)], axis=0))

y_train_enc = le.transform(y_train.astype(str))
y_test_enc  = le.transform(y_test.astype(str))

configurations.append((f'PCA', X_train_pca, X_test_pca,y_train_enc))
configurations.append((f'LDA', X_train_lda, X_test_lda, y_train_enc))


# Step 7: Run XGBoosting on different configurations
print("\n=== XGBoost Model Performance ===")

xgb = XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    tree_method="hist",
    n_jobs=-1
)

for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning XGBoosting with {name} configuration...")

    xgb.fit(X_train_cfg, y_train_cfg)

    y_train_pred = xgb.predict(X_train_cfg)
    y_test_pred  = xgb.predict(X_test_cfg)

    y_train_proba = xgb.predict_proba(X_train_cfg)
    y_test_proba  = xgb.predict_proba(X_test_cfg)


    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_pred),
            metrics.accuracy_score(y_test_enc,      y_test_pred),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.f1_score(y_test_enc,      y_test_pred,  average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.recall_score(y_test_enc,      y_test_pred,  average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.precision_score(y_test_enc,      y_test_pred,  average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test_enc),      y_test_proba,  multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nXGBoosting Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_proba, multi_class='ovr', average='macro')
    storeResults('XGBoosting', name,
                 metrics.accuracy_score(y_test_enc, y_test_pred),
                 metrics.f1_score(y_test_enc, y_test_pred, average='macro'),
                 metrics.recall_score(y_test_enc, y_test_pred, average='macro'),
                 metrics.precision_score(y_test_enc, y_test_pred, average='macro'),
                 auc_score)




=== XGBoost Model Performance ===

Running XGBoosting with PCA configuration...

XGBoosting Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  0.999813  0.999571 0.999864   0.999279 1.000000
    Test  0.997887  0.970168 0.969220   0.971175 0.999905

Running XGBoosting with LDA configuration...

XGBoosting Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  0.989344  0.951134 0.944464   0.959344 0.999775
    Test  0.984889  0.905786 0.889268   0.932112 0.999316


#CatBoosting

In [ ]:
# Configuration list to store different data setups
configurations = []

configurations.append((f'PCA', X_train_pca, X_test_pca, y_train))
configurations.append((f'LDA', X_train_lda, X_test_lda, y_train))


# Step 7: Run CatBoosting on different configurations
print("\n=== CatBoost Model Performance ===")
cat = CatBoostClassifier(


    bagging_temperature =0.05,
    boosting_type = 'Plain',
    learning_rate=0.05,
    depth=3,
    n_estimators=100,
    random_seed=42,
    silent =True

)

for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning CatBoosting with {name} configuration...")

    cat.fit(X_train_cfg, y_train_cfg)

    y_train_pred = cat.predict(X_train_cfg)
    y_test_pred  = cat.predict(X_test_cfg)

    y_train_proba = cat.predict_proba(X_train_cfg)
    y_test_proba  = cat.predict_proba(X_test_cfg)


    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_pred),
            metrics.accuracy_score(y_test,      y_test_pred),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.f1_score(y_test,      y_test_pred,  average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.recall_score(y_test,      y_test_pred,  average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.precision_score(y_test,      y_test_pred,  average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test),      y_test_proba,  multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nCatBoosting Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_proba, multi_class='ovr', average='macro')
    storeResults('CatBoosting', name,
                 metrics.accuracy_score(y_test, y_test_pred),
                 metrics.f1_score(y_test, y_test_pred, average='macro'),
                 metrics.recall_score(y_test, y_test_pred, average='macro'),
                 metrics.precision_score(y_test, y_test_pred, average='macro'),
                 auc_score)


=== CatBoost Model Performance ===

Running CatBoosting with PCA configuration...

CatBoosting Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  0.949745  0.714573 0.712716   0.716611 0.993252
    Test  0.949946  0.715118 0.714248   0.716221 0.993899

Running CatBoosting with LDA configuration...

CatBoosting Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  0.930933  0.703927 0.716191   0.694327 0.992479
    Test  0.931666  0.704764 0.716549   0.695598 0.992742


#Bagging

In [ ]:
# Configuration list to store different data setups
configurations = []

configurations.append((f'PCA', X_train_pca, X_test_pca, y_train))
configurations.append((f'LDA', X_train_lda, X_test_lda, y_train))

# Step 7: Run Bagging Classifier on different configurations
print("\n=== Bagging Model Performance ===")

bag = BaggingClassifier(
        estimator=DecisionTreeClassifier(max_depth=5, random_state=42),
        n_estimators=200,
        max_samples=1.0,
        max_features=1.0,
        bootstrap=True,
        bootstrap_features=False,
        n_jobs=-1,
        random_state=42
    )
for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning Bagging with {name} configuration...")

    bag.fit(X_train_cfg, y_train_cfg)

    y_train_pred = bag.predict(X_train_cfg)
    y_test_pred  = bag.predict(X_test_cfg)

    y_train_proba = bag.predict_proba(X_train_cfg)
    y_test_proba  = bag.predict_proba(X_test_cfg)


    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_pred),
            metrics.accuracy_score(y_test,      y_test_pred),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.f1_score(y_test,      y_test_pred,  average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.recall_score(y_test,      y_test_pred,  average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_pred, average='macro'),
            metrics.precision_score(y_test,      y_test_pred,  average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test),      y_test_proba,  multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nBagging Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_proba, multi_class='ovr', average='macro')
    storeResults('Bagging Classifier', name,
                 metrics.accuracy_score(y_test, y_test_pred),
                 metrics.f1_score(y_test, y_test_pred, average='macro'),
                 metrics.recall_score(y_test, y_test_pred, average='macro'),
                 metrics.precision_score(y_test, y_test_pred, average='macro'),
                 auc_score)




=== Bagging Model Performance ===

Running Bagging with PCA configuration...

Bagging Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  0.946353  0.709919 0.720889   0.702906 0.990496
    Test  0.946756  0.709147 0.720151   0.702007 0.991983

Running Bagging with LDA configuration...

Bagging Model Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  0.935977  0.716155 0.734860   0.701037 0.992353
    Test  0.936322  0.716661 0.734418   0.702346 0.991928


# Voting Classifier

In [ ]:
# Configuration list to store different data setups
configurations = []

le = LabelEncoder()
# Fit on the union to avoid unseen class errors if test has a class not in train
le.fit(pd.concat([y_train.astype(str), y_test.astype(str)], axis=0))

y_train_enc = le.transform(y_train.astype(str))
y_test_enc  = le.transform(y_test.astype(str))

configurations.append((f'PCA', X_train_pca, X_test_pca,y_train_enc))
configurations.append((f'LDA', X_train_lda, X_test_lda, y_train_enc))


# Step 7: Run Voting Classifier on different configurations
print("\n=== Voting Model Performance ===")

# base learners
rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1
)

knn = KNeighborsClassifier(
    n_neighbors=11,
    weights="distance",
    metric="minkowski",
    p=2,
    n_jobs=-1
)

xgb = XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    tree_method="hist",
    n_jobs=-1
)

for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nTraining models with {name} configuration...")

    rf.fit(X_train_cfg, y_train_cfg)
    knn.fit(X_train_cfg, y_train_cfg)
    xgb.fit(X_train_cfg, y_train_cfg)

    # Define y_test_cfg for each configuration
    y_test_cfg = y_test_enc
    # Voting configurations using the actual model objects
    # === Voting classifier configurations ===
    voting_configs = [
       ("hard", VotingClassifier(estimators=[("rf", rf), ("knn", knn), ("xgb", xgb)], voting="hard")),
       ("soft", VotingClassifier(estimators=[("rf", rf), ("knn", knn), ("xgb", xgb)], voting="soft")),
       ("weighted_hard", VotingClassifier(estimators=[("rf", rf), ("knn", knn), ("xgb", xgb)], voting="hard", weights=[0.3, 0.3, 0.4])),
       ("weighted_soft", VotingClassifier(estimators=[("rf", rf), ("knn", knn), ("xgb", xgb)], voting="soft", weights=[0.4, 0.3, 0.3]))
    ]

    for voting_type, voting_clf in voting_configs:
        print(f"\n=== Voting Classifier ({voting_type}) with {name} ===")

        # Fit voting classifier
        voting_clf.fit(X_train_cfg, y_train_cfg)

        # Predictions
        y_train_pred = voting_clf.predict(X_train_cfg)
        y_test_pred = voting_clf.predict(X_test_cfg)

        # Check if voting is hard or soft for probabilities
        if 'hard' in voting_type:
            # For hard voting, we cannot get probabilities, so set AUC-ROC to 0
            auc_train = 0
            auc_test = 0
        else:
            # For soft voting, we can get probabilities
            y_train_proba = voting_clf.predict_proba(X_train_cfg)
            y_test_proba = voting_clf.predict_proba(X_test_cfg)
            auc_train = metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_proba, multi_class='ovr', average='macro')
            auc_test = metrics.roc_auc_score(pd.get_dummies(y_test_cfg), y_test_proba, multi_class='ovr', average='macro')

        # Calculate metrics
        metrics_dict = {
            "Dataset": ["Training", "Test"],
            "Accuracy": [
                metrics.accuracy_score(y_train_cfg, y_train_pred),
                metrics.accuracy_score(y_test_cfg, y_test_pred),
            ],
            "F1 Score": [
                metrics.f1_score(y_train_cfg, y_train_pred, average='macro'),
                metrics.f1_score(y_test_cfg, y_test_pred, average='macro'),
            ],
            "Recall": [
                metrics.recall_score(y_train_cfg, y_train_pred, average='macro'),
                metrics.recall_score(y_test_cfg, y_test_pred, average='macro'),
            ],
            "Precision": [
                metrics.precision_score(y_train_cfg, y_train_pred, average='macro'),
                metrics.precision_score(y_test_cfg, y_test_pred, average='macro'),
            ],
            "AUC-ROC": [auc_train, auc_test]
        }

        df_metrics = pd.DataFrame(metrics_dict)
        print(f"\nVoting Classifier ({voting_type}) Performance Metrics")
        print(df_metrics.to_string(index=False))

        storeResults(
            f'Voting Classifier ({voting_type})',
            name,
            metrics.accuracy_score(y_test_cfg, y_test_pred),
            metrics.f1_score(y_test_cfg, y_test_pred, average='macro'),
            metrics.recall_score(y_test_cfg, y_test_pred, average='macro'),
            metrics.precision_score(y_test_cfg, y_test_pred, average='macro'),
            auc_test
        )



=== Voting Model Performance ===

Training models with PCA configuration...

=== Voting Classifier (hard) with PCA ===

Voting Classifier (hard) Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  1.000000  1.000000 1.000000   1.000000        0
    Test  0.998168  0.972309 0.965323   0.980106        0

=== Voting Classifier (soft) with PCA ===

Voting Classifier (soft) Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  1.000000  1.000000 1.000000   1.000000 1.000000
    Test  0.998081  0.971986 0.965221   0.979567 0.999933

=== Voting Classifier (weighted_hard) with PCA ===

Voting Classifier (weighted_hard) Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  1.000000  1.000000 1.000000   1.000000        0
    Test  0.998232  0.972394 0.965455   0.980144        0

=== Voting Classifier (weighted_soft) with PCA ===

Voting Classifier (weighted_soft) Performance Metrics
 Dataset 

# Stacking

In [ ]:
# Configuration list to store different data setups
configurations = []

le = LabelEncoder()
# Fit on the union to avoid unseen class errors if test has a class not in train
le.fit(pd.concat([y_train.astype(str), y_test.astype(str)], axis=0))

y_train_enc = le.transform(y_train.astype(str))
y_test_enc  = le.transform(y_test.astype(str))

configurations.append((f'PCA', X_train_pca, X_test_pca,y_train_enc))
configurations.append((f'LDA', X_train_lda, X_test_lda, y_train_enc))


# Step 7: Run Stacking Classifier on different configurations
print("\n=== Stacking Model Performance ===")

# base learners
rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1
)

knn = KNeighborsClassifier(
    n_neighbors=11,
    weights="distance",
    metric="minkowski",
    p=2,
    n_jobs=-1
)

cat = CatBoostClassifier(
    learning_rate=0.05,
    depth=3,
    n_estimators=100,
    bagging_temperature=0.05,
    boosting_type='Plain',
    random_seed=42,
    verbose=False
)

# Train each model
for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nTraining models with {name} configuration...")

    rf.fit(X_train_cfg, y_train_cfg)
    knn.fit(X_train_cfg, y_train_cfg)
    cat.fit(X_train_cfg, y_train_cfg)

    # Define y_test_cfg for each configuration
    y_test_cfg = y_test_enc
    # Create stacking classifier
    base_estimators = [
        ('rf', rf),
        ('knn', knn),
        ('cat', cat)
    ]

    # Meta-learner
    meta_learner = LogisticRegression(max_iter=1000, random_state=42)

    # Stacking classifier
    stacking_clf = StackingClassifier(
        estimators=base_estimators,
        final_estimator=meta_learner,
        cv=5,  # Use 5-fold cross-validation to train meta-learner
        stack_method='predict_proba',  # Use probabilities for meta-features
        n_jobs=-1
    )

    # Fit stacking classifier
    stacking_clf.fit(X_train_cfg, y_train_cfg)

    # Predictions
    y_train_pred = stacking_clf.predict(X_train_cfg)
    y_test_pred = stacking_clf.predict(X_test_cfg)
    y_train_proba = stacking_clf.predict_proba(X_train_cfg)
    y_test_proba = stacking_clf.predict_proba(X_test_cfg)

    # Calculate metrics
    # Calculate metrics
    metrics_dict = {
            "Dataset": ["Training", "Test"],
            "Accuracy": [
                metrics.accuracy_score(y_train_cfg, y_train_pred),
                metrics.accuracy_score(y_test_cfg, y_test_pred),
            ],
            "F1 Score": [
                metrics.f1_score(y_train_cfg, y_train_pred, average='macro'),
                metrics.f1_score(y_test_cfg, y_test_pred, average='macro'),
            ],
            "Recall": [
                metrics.recall_score(y_train_cfg, y_train_pred, average='macro'),
                metrics.recall_score(y_test_cfg, y_test_pred, average='macro'),
            ],
            "Precision": [
                metrics.precision_score(y_train_cfg, y_train_pred, average='macro'),
                metrics.precision_score(y_test_cfg, y_test_pred, average='macro'),
            ],
            "AUC-ROC": [auc_train, auc_test]
        }

    df_metrics = pd.DataFrame(metrics_dict)
    print(f"\nStacking Classifier Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test_cfg), y_test_proba, multi_class='ovr', average='macro')

    storeResults(
           'Stacking Classifier', name,
            metrics.accuracy_score(y_test_cfg, y_test_pred),
            metrics.f1_score(y_test_cfg, y_test_pred, average='macro'),
            metrics.recall_score(y_test_cfg, y_test_pred, average='macro'),
            metrics.precision_score(y_test_cfg, y_test_pred, average='macro'),
            auc_test
        )



=== Stacking Model Performance ===

Training models with PCA configuration...

Stacking Classifier Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  1.000000  1.000000 1.000000    1.00000   1.0000
    Test  0.998146  0.960632 0.955924    0.96576   0.9998

Training models with LDA configuration...

Stacking Classifier Performance Metrics
 Dataset  Accuracy  F1 Score   Recall  Precision  AUC-ROC
Training  1.000000  1.000000 1.000000   1.000000   1.0000
    Test  0.994891  0.966255 0.961412   0.971545   0.9998


# Data Frame

In [ ]:
# Creating the dataframe

# Make sure output folder exists
out_dir = Path("final_results")
out_dir.mkdir(parents=True, exist_ok=True)

result = pd.DataFrame({
    'ML Model': ML_Model,
    'Configuration': ML_Config,
    'Accuracy': [f"{acc * 100:.3f}%" for acc in accuracy],
    'F1 Score': [f"{f1 * 100:.3f}%" for f1 in f1_score],
    'Recall': [f"{rec * 100:.3f}%" for rec in recall],
    'Precision': [f"{prec * 100:.3f}%" for prec in precision],
    'ROC_AUC': [f"{roc * 100:.3f}%" for roc in auc_roc],
})

# Remove duplicates based on model and configuration
result.drop_duplicates(subset=["ML Model", "Configuration"], inplace=True)

# Display the result
print("\n" + "=" * 100)
print("MODEL PERFORMANCE RESULTS")
print("=" * 100)
print(result.to_string(index=False))

# Save the result to a CSV file
result.to_csv('final_results/model_results.csv', index=False)
print("\nResults saved to model_results.csv")

# Sort by Accuracy and F1 Score
sorted_result = result.sort_values(by=['Accuracy', 'F1 Score'], ascending=False).reset_index(drop=True)

# Display the sorted result
print("\n" + "=" * 100)
print("SORTED MODEL PERFORMANCE RESULTS (by Accuracy and F1 Score)")
print("=" * 100)
print(sorted_result.to_string(index=False))

# Save the sorted result
sorted_result.to_csv('final_results/sorted_model_results.csv', index=False)
print("\nSorted results saved to sorted_model_results.csv")

# Extract top configuration per ML model
top_per_model = sorted_result.groupby('ML Model', as_index=False).first()

# Display and save the top configuration table
print("\n" + "=" * 100)
print("TOP CONFIGURATION PER MODEL")
print("=" * 100)
print(top_per_model.to_string(index=False))

top_per_model.to_csv('final_results/top_configurations.csv', index=False)
print("\nTop configuration per model saved to top_configurations.csv")


MODEL PERFORMANCE RESULTS
                         ML Model Configuration Accuracy F1 Score  Recall Precision ROC_AUC
           Support Vector Machine           PCA  98.709%  77.373% 78.234%   76.586% 99.830%
           Support Vector Machine           LDA  95.570%  73.736% 74.755%   72.833% 99.413%
                    Random Forest           PCA  99.823%  96.674% 95.635%   97.898% 99.990%
                    Random Forest           LDA  99.455%  96.768% 96.105%   97.512% 99.748%
                              KNN           PCA  99.737%  97.078% 96.441%   97.798% 99.488%
                              KNN           LDA  99.368%  95.882% 95.168%   96.724% 99.430%
                Gradient Boosting           PCA  99.064%  87.878% 86.355%   90.804% 98.000%
                Gradient Boosting           LDA  96.642%  84.033% 82.327%   88.044% 95.818%
                      AdaBoosting           PCA  89.049%  61.928% 59.247%   66.140% 97.047%
                      AdaBoosting           LDA  90.2